In [96]:
input_text=[]
target_text=[]
input_char=set()
target_char=set()
with open ('mar.txt','r',encoding='utf-8') as f:
  lines=f.read().split('\n')


In [4]:
num_samples=3500

In [97]:
for sent in text[:min(num_samples,len(text)-1)] :
  input_texts,target_texts,_r=sent.split('\t')
  input_text.append(input_texts)
  target_texts='\t'+target_texts+'\n'
  target_text.append(target_texts)
  for char in input_texts:
    input_char.add(char)
  for char in target_texts:
    target_char.add(char)

  


In [ ]:
input_char=sorted(list(input_char))
target_char=sorted(list(target_char))
encoder_len=len(input_char)
decoder_len=len(target_char)
max_len_encoder=max([len(tex) for tex in input_text])
max_len_decoder=max([len(tex) for tex in target_text])

In [ ]:
input_dict={}
target_dict={}
for i,char in enumerate(input_char):
  input_dict[char]=i
for i,char in enumerate(target_char):
  target_dict[char]=i

In [ ]:
import numpy as np

In [ ]:
encoder_input=np.zeros((len(input_text),max_len_encoder,encoder_len),dtype='float32')
decoder_input=np.zeros((len(target_text),max_len_decoder,decoder_len),dtype='float32')
decoder_target=np.zeros((len(target_text),max_len_decoder,decoder_len),dtype='float32')

In [ ]:
for i in range(len(input_text)):
  for j,char in enumerate(input_text[i]):
    encoder_input[i,j,input_dict[char]]=1
  encoder_input[i,j+1:,input_dict[' ']]=1

In [ ]:
for i in range(len(target_text)):
  for j,char in enumerate(target_text[i]):
    decoder_input[i,j,target_dict[char]]=1
    if j>0 :
      decoder_target[i,j-1,target_dict[char]]=1
  decoder_input[i,j+1:,target_dict[' ']]=1
  decoder_target[i,j:,target_dict[' ']]=1
  

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense

In [ ]:
encoder_inputs = Input(shape=(None, encoder_len))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, decoder_len))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(decoder_len, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [65]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input, decoder_input], decoder_target,
          batch_size=64,
          epochs=100,
          validation_split=0.2)

Epoch 1/100
88/88 [==============================] - 27s 269ms/step - loss: 2.0893 - val_loss: 1.6878
Epoch 2/100
88/88 [==============================] - 22s 254ms/step - loss: 1.4058 - val_loss: 1.4481
Epoch 3/100
88/88 [==============================] - 22s 253ms/step - loss: 1.1539 - val_loss: 1.2405
Epoch 4/100
88/88 [==============================] - 22s 252ms/step - loss: 0.9403 - val_loss: 1.0660
Epoch 5/100
88/88 [==============================] - 22s 253ms/step - loss: 0.8370 - val_loss: 0.9975
Epoch 6/100
88/88 [==============================] - 22s 252ms/step - loss: 0.7667 - val_loss: 0.9538
Epoch 7/100
88/88 [==============================] - 22s 252ms/step - loss: 0.7188 - val_loss: 0.9237
Epoch 8/100
88/88 [==============================] - 22s 253ms/step - loss: 0.6665 - val_loss: 0.8907
Epoch 9/100
88/88 [==============================] - 22s 253ms/step - loss: 0.6115 - val_loss: 0.8838
Epoch 10/100
88/88 [==============================] - 22s 255ms/step - loss: 0.570

In [67]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [92]:
reverse_target_char_index=dict((i,char) for char,i in target_dict.items())



In [93]:
def decode_sequence(input_seq):

    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, decoder_len))
    target_seq[0, 0, target_dict['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or
           len(decoded_sentence) > decoder_len):
            stop_condition = True
        target_seq = np.zeros((1, 1, decoder_len))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence

In [95]:
for seq_index in range(100):
  input_seq=encoder_input[seq_index:seq_index+1]
  decoded_sentence=decode_sequence(input_seq)
  print('input text:',input_text[seq_index])
  print('Translation to Marathi',decoded_sentence)


input text: Go.
Translation to Marathi जा.

input text: Go.
Translation to Marathi जा.

input text: Run!
Translation to Marathi 	धाव!

input text: Run!
Translation to Marathi 	धाव!

input text: Run!
Translation to Marathi 	धाव!

input text: Run!
Translation to Marathi 	धाव!

input text: Run!
Translation to Marathi 	धाव!

input text: Run!
Translation to Marathi 	धाव!

input text: Run!
Translation to Marathi 	धाव!

input text: Run!
Translation to Marathi 	धाव!

input text: Who?
Translation to Marathi कोण?

input text: Who?
Translation to Marathi कोण?

input text: Wow!
Translation to Marathi वाह!

input text: Wow!
Translation to Marathi वाह!

input text: Fire!
Translation to Marathi 	आग!

input text: Fire!
Translation to Marathi 	आग!

input text: Fire!
Translation to Marathi 	आग!

input text: Fire!
Translation to Marathi 	आग!

input text: Help!
Translation to Marathi 	वाचवा!

input text: Help!
Translation to Marathi 	वाचवा!

input text: Help!
Translation to Marathi 	वाचवा!

input text: He